# Experiment tracking with TensorBoard

NVIDIA FLARE uses `TBAnalyticsReceiver` for experiment tracking on the FL server by default, enabling experiment tracking.

#### Install requirements
Make sure to install the required packages:

In [ ]:
% pip install -r code/requirements.txt

## The `TBAnalyticsReceiver`

The FedJob API makes it easy to create job configurations, and by default the `TBAnalyticsReceiver` for TensorBoard streaming is included. You can specify your own analytics_receiver of type `AnalyticsReceiver` as a parameter if you want, but if left unspecified, `TBAnalyticsReceiver` is configured to be set up in `BaseFedJob` for [Pytorch](https://github.com/NVIDIA/NVFlare/blob/main/nvflare/app_opt/pt/job_config/base_fed_job.py#L98) and for [Tensorflow](https://github.com/NVIDIA/NVFlare/blob/main/nvflare/app_opt/tf/job_config/base_fed_job.py#L95) APIs.

The `TBAnalyticsReceiver` for TensorBoard streaming receives and records the logs during the experiment by saving them to Tensoboard event files on the FL server. 

See [this link](https://nvflare.readthedocs.io/en/main/programming_guide/experiment_tracking/experiment_tracking_log_writer.html#tools-sender-logwriter-and-receivers) for more details on the other available AnalyticsReceivers in NVFlare: MLflowReceiver and WandBReceiver.

## Add SummaryWriter and `add_scalar` for logging metrics

To keep things simple, we start from the state of the code we had in [section 1.3](../..//01.3_server_side_customization/customize_server_logics.ipynb) of this chapter, and make a few modifications needed to implement metrics logging for experiment tracking.

### Create SummaryWriter and log metrics

In order to add SummaryWriter to the client training code, we need to import it with the following line (at the top of [client.py](code/src/client.py)):

```python
from nvflare.client.tracking import SummaryWriter
```

After that, we need to add the following line after `flare.init()`:
```python
summary_writer = SummaryWriter()
```

We can then use `summary_writer` to log any experimental metrics. In this case, we have a local_accuracy available already, so we can use `add_scalar()` to log this:

```python
summary_writer.add_scalar(tag="local_accuracy", scalar=local_accuracy, global_step=global_step)
```

Note that the `global_step` is included here, which we calculate on the previous line:

```python
global_step = input_model.current_round * n_loaders + i
```

You can see the full contents of the updated training code in [client.py](code/src/client.py):

In [ ]:
!cat code/src/client.py

Let's run this example by executing the following command. The num_rounds has been increased to 20 in order to have more data for a better looking graph.

In [ ]:
! cd code && python fl_job.py

## View tensorboard results


In order to see the results, you can use the following command directed to the location of the TensorBoard event files (by default, the location for the server should be as follows using the default simulator path provided):


```commandline
tensorboard --logdir=/tmp/nvflare/jobs/workdir/server/simulate_job/tb_events
```

In [ ]:
%load_ext tensorboard

In [ ]:
%tensorboard --logdir=/tmp/nvflare/jobs/workdir/server/simulate_job/tb_events --bind_all

Now we know how experiment tracking can be achieved through metric logging and can be configured to work in a job with an `AnalyticsReceiver`. With this mechanism, we can stream various types of metric data.


For how to use `MLflowReceiver` to set up experiment tracking for MLflow, see [Experiment Tracking with MLflow](../01.5.2_experiment_tracking_with_mlflow/experiment_tracking_mlflow.ipynb).